# DataJoint Calcium Imaging Quality Metrics

Visualize the calcium imaging quality metrics for the raw and processed scans that are stored in the DataJoint pipeline (i.e. `element-calcium-imaging`).

If you are new to using this DataJoint pipeline for analyzing calcium imaging data, please see the [tutorial](./tutorial.ipynb) notebook for an in-depth explanation to set up and run the workflow.

This notebook can run in a [GitHub Codespace](https://github.com/datajoint/workflow-calcium-imaging#interactive-tutorial), and requires the example data to be populated into the database using the [demo_prepare](./demo_prepare.ipynb) notebook.

## 1. Scan Quality Metrics

### 1.1 Frame Metrics
Frame metrics are quantitative measurements used to evaluate the consistency of imaging for each individual frame over time. There are several parameters to consider, such as the minimum intensity, mean intensity, maximum intensity, and contrast of the calcium imaging frames.

**min_intensity:** The minimum intensity is the lowest fluorescence intensity value recorded during the imaging session. It reflects the ADC bias and dark current of the imaging system since many pixels receive no photons. Changes in minimum intensity could indicate various issues, such as changes in imaging conditions or technical issues with the imaging equipment or software. In either case, it may be necessary to adjust the imaging conditions or troubleshoot the equipment or software to identify and correct the problem.

**mean_intensity:** Changes in mean intensity could be caused by changes in imaging conditions or photobleaching, which is a decrease in mean intensity due to the irreversible damage of fluorescent molecules in the sample from light exposure. Adjusting the imaging conditions to ensure consistency over time or reducing the light exposure can prevent photobleaching.

**max_intensity:** The maximum intensity is the highest fluorescence intensity value recorded during the imaging session. Changes in maximum intensity could result from changes in imaging conditions, changes in sample properties, photobleaching, or saturation of the camera or detector. Depending on the cause of the changes, it may be necessary to adjust the imaging protocol, control for sample changes, or reduce the light exposure.

**contrast:** Contrast refers to the difference in brightness or intensity between the brightest and darkest parts of an image. High contrast images have a large difference between the brightest and darkest parts, while low contrast images have a small difference. If the contrast of calcium imaging frames changes over time, it can have implications for the interpretation of the data. A decrease in contrast may make it harder to detect calcium signals accurately, while changes in contrast between different imaging sessions may make it harder to compare images over time.

## 2. Processing Quality Metrics

### 2.1 Mask
**mask_area:** Mask area is a quality control metric commonly used in image segmentation processes for cell detection (see. [Stringer & Pachitariu, 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6525101/)). In cell detection, the goal is to identify and segment individual cells within an image or set of images. This process involves identifying and isolating the pixels that belong to each cell, and creating a mask that represents the boundaries of each cell. The mask is a binary image that indicates which pixels belong to the cell and which pixels do not. The mask area refers to the total number of pixels in the mask that represent the cell.

In quality control for cell detection, the mask area is used as a metric to evaluate the accuracy and consistency of the segmentation process. Specifically, the mask area can be compared to the expected area of a single cell, which may be determined from prior knowledge or reference data. If the mask area is significantly different from the expected area, it may indicate errors or inconsistencies in the segmentation process.

For example, if the mask area is much larger than expected for a single cell, it may indicate that the segmentation process is including pixels that do not belong to the cell, leading to false positives. This can result from a variety of factors, including variations in staining, illumination, or noise. On the other hand, if the mask area is much smaller than expected, it may indicate that the segmentation process is missing pixels that belong to the cell, leading to false negatives.

By monitoring the mask area as a quality control metric, image segmentation processes for cell detection can be improved to ensure that they are accurate and consistent. This can involve adjusting parameters, such as threshold values or filter settings, or using more advanced segmentation techniques, such as deep learning approaches.

**roundness:** Roundness is another quality control metric used to evaluate the shape of segmented cells ([Tegtmeier et al., 2018](https://www.frontiersin.org/articles/10.3389/fnins.2018.00958/full)). Roundness measures how closely a cell's shape resembles a perfect circle. It is calculated as the ratio of the cell's actual area to the area of a circle with the same perimeter. A perfectly round cell will have a roundness value of 1, while more elongated or irregular cells will have lower roundness values.

Roundness is a useful metric in cell detection because it can help identify cells that have been improperly segmented. For example, cells that are elongated or irregular in shape may have been segmented incorrectly due to noise, overlapping cells, or other factors. By comparing the roundness of segmented cells to a threshold value, cells that are improperly segmented can be identified and corrected.

### 2.2 Trace
**skewness ([Stringer & Pachitariu, 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6525101/)):** The skewness of fluorescence activity from multi-day long experiments can be used as a quality control metric by providing an indication of the consistency and stability of the signal over time. A stable signal with consistent skewness values across multiple days suggests that the measurement system is working properly and the experimental conditions are consistent. However, changes in the skewness of fluorescence activity over time could indicate technical issues or changes in the sample physiology or behavior, which could affect the quality of the data and the accuracy of any conclusions drawn from the experiment.

In vivo measurements may not necessarily show the same skewness as in vitro or ex vivo measurements, as the complexity and variability of the in vivo environment can introduce additional factors that may affect the fluorescence signal. For example, changes in the animal's behavior, stress levels, or physiological state could affect the skewness of the fluorescence activity over time. Therefore, it is important to establish appropriate baseline measurements and controls for in vivo experiments, and to carefully interpret any changes in skewness within the context of the experimental conditions and the animal's behavior and physiology.

**variance ([Stringer & Pachitariu, 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6525101/)):** The variance of fluorescence activity from multi-day long experiments can be used as a quality control metric by providing an indication of the consistency of the signal over time. A high variance could indicate that the signal is unstable, and there are large fluctuations in the fluorescence activity between different imaging sessions. On the other hand, a low variance suggests that the signal is stable and consistent over time.

To use variance as a quality control metric, researchers can calculate the variance of the fluorescence activity across frames and compare it to a threshold value. If the variance exceeds the threshold, further investigation may be required to identify potential sources of instability in the imaging setup or data processing.

In vivo measurements may have different variance values compared to in vitro measurements, as the biological system in vivo is more complex and dynamic. Variations in the activity of neurons, changes in the animal's behavior, and other physiological factors can contribute to the variability of the signal. Therefore, it is important to establish appropriate thresholds for variance based on the specific experimental setup and context.


In [ ]:
import datajoint as dj
from workflow_calcium_imaging.pipeline import subject, session, scan, imaging